In [ ]:
import tensorflow as tf
import cv2
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
from tensorflow.keras.applications.inception_v3 import preprocess_input
image_dir = "D:/aircraft dataset/Images"
#from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# Path to the directory containing the XML annotations
annotation_dir = 'D:/aircraft dataset/Annotations/Horizontal Bounding Boxes'


# List to store the image paths and annotations
croped_images = []
#bounding_boxes = []
cropedd_labels = []
zeros = []
# Iterate over the XML files in the annotation directory
for filename in os.listdir(annotation_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(annotation_dir, filename)
        image_filename = os.path.splitext(filename)[0] + '.jpg'


        # Read the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Extract the image size information
        size_element = root.find('size')
        width = int(size_element.find('width').text)
        height = int(size_element.find('height').text)
        if width == 0 or height == 0:
            zeros.append(image_filename)
            continue
        image_path = os.path.join(image_dir,"JPEGImages", image_filename)
        image_path = image_path.replace('\\', '/')
        image = cv2.imread(image_path)
        
        print(image_path)
        # Extract annotation information for each object
        
        class_label = obj.find('name').text

        # Append the bounding box coordinates to the list

        if image_croped.size == 0:
            print(image_path)
            continue
        new_image_size = (120, 120)
        image = cv2.resize(image, new_image_size, interpolation=cv2.INTER_LINEAR)  # Resize the image to match the input size of InceptionV3
        image = preprocess_input(np.array(image))

        croped_images.append(image)
        cropedd_labels.append(int(class_label[1:]))

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
  
import numpy as np
import matplotlib.pyplot as plt
# Assuming `images_array` is the NumPy array of images and `labels` is the corresponding array of labels

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(croped_images, cropedd_labels, test_size=0.2, random_state=42)
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels) 
test_labels = np.array(test_labels)
# Check the shapes of the split datasets
print("Training images shape:", train_images.shape)
print("Testing images shape:", test_images.shape)
print("Training labels shape:", train_labels.shape)
print("Testing labels shape:", test_labels.shape)




In [ ]:

def build_model():
    data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",
                      input_shape=(64,64,3)),
    tf.keras.layers.RandomZoom(0.07),
    tf.keras.layers.RandomRotation(180) 
  ]
)
    model = tf.keras.models.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(450, activation='relu'),
    tf.keras.layers.Dropout(0.65),
    tf.keras.layers.Dense(10, activation='softmax')
])
   
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=opt,
                    metrics=['acc'])
    print(model.summary())
    return model
model = build_model()

In [ ]:
epochs = 30
x = np.arange(epochs) + 1
history_ = model.fit(x_train, y_train, epochs=epochs, validation_split=.20, batch_size=64)
plt.plot(x,history_.history['acc'])
plt.plot(x,history_.history['val_acc'])
plt.xlabel("Epochs");
plt.ylabel("Accuracy")
plt.legend(['Training Accuacy', 'Cross Validation Accuracy']);

fig, ax = plt.subplots()
ax.plot(x,history_.history['loss'])
ax.plot(x,history_.history['val_loss'])
plt.xlabel("Epochs");
plt.ylabel("Loss")
plt.legend(['Training Loss', 'Cross Validation Loss']);


In [ ]:
!pip install torch torchvision
!pip install yolov5


In [ ]:

import torch
from PIL import Image
from matplotlib import pyplot as plt
from IPython.display import display
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.pt')

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from PIL import Image
from tensorflow.keras.applications.inception_v3 import preprocess_input

import matplotlib.patches as patches
image_path = 'D:/aircraft dataset/Images/JPEGImages/13.jpg'
image = Image.open(image_path)
#image =np.array( image)
# Perform detection
# Set the class label for aircraft
# Set the desired class number
desired_class = 4
# Perform detection
results = model(image)
# Filter the results based on the class number
filtered_results = results.pred[0][results.pred[0][:, -1] == desired_class]

# Access the bounding box coordinates
bboxes = filtered_results[:, :4]
# Access the bounding box coordinates and class labels
#bboxes = filtered_results.xyxy[0].numpy()[:, :4]


# Enable inline plotting in Google Colab
%matplotlib inline

fig, ax = plt.subplots()

# Hide the axes and labels
ax.axis('off')

# Display the image
ax.imshow(image)

# Create a rectangle patch for each bounding box and add it to the axes
for box in bboxes:
    # Extract the box coordinates
    x_min, y_min, x_max, y_max = box
    print(box)
    # Create a rectangle patch
    rect = patches.Rectangle((int(x_min), int(y_min)), int(x_max) - int(x_min), int(y_max) - int(y_min),
                             linewidth=1, edgecolor='r', facecolor='none')

    # Add the rectangle patch to the axes
    ax.add_patch(rect)

# Show the plot
plt.show()

In [ ]:
model.save()


In [ ]:
model = tf.saved_model.load('C:/Users/Тетяна/Desktop')
for box in bboxes:
    # Extract the box coordinates
    xmin, ymin, xmax, ymax = box
    #image =Image.open(image_path)
    
    #image_croped = image[ymin:ymax, xmin:xmax]
    image_croped = image.crop((int(xmin.item()), int(ymin.item()), int(xmax.item()), int(ymax.item())))
    image_croped = cv2.resize(image_croped, (130,130), interpolation=cv2.INTER_LINEAR)  # Resize the image to match the input size of InceptionV3
    image_croped = preprocess_input(np.array(image_croped))
    results = model(image)
    print(results)